# UltraSonicSonar
Here is an ultrasonic sonar class that I created to use with ROS on my RaspberryPi computer.
It works with *HC-SR04* ultrasonic sonar. You can try to use other ultrasonic sonars because they are all based on the same principle.

In [4]:
#!/usr/bin/env python
import math
import RPi.GPIO as GPIO
import time

SONIC_SPEED = 343.0 # sonic speed m/s
HALF_SONIC_SPEED = SONIC_SPEED / 2.0

class UltraSonicSonar():
    def __init__(self, name:str, gpio_trigger:int, gpio_echo:int, min_range:float, max_range:float, \
                 field_of_view:int = math.nan):
        """
        sonar name, trigger GPIO, echo GPIO, min and max range, field of view
        """
        self.__name = name
        self.__gpio_trigger = gpio_trigger
        self.__gpio_echo = gpio_echo
        self.__min_range = min_range
        self.__max_range = max_range
        if not math.isnan(field_of_view):
            self.__field_of_view = math.radians(field_of_view)  # in radians
        else:
            self.__field_of_view = math.nan
        # echo maximum waiting time. rounded up to 3 decimal places
        self.__max_wait_time = self.__max_wait_time = math.ceil(max_range / SONIC_SPEED * 1000) / 1000

    def init_hw(self):
        """initialize hardware"""
        # set GPIO mode to BCM
        GPIO.setmode(GPIO.BCM)

        GPIO.setup(self.__gpio_trigger, GPIO.OUT)
        GPIO.output(self.__gpio_trigger, GPIO.LOW)
        GPIO.setup(self.__gpio_echo, GPIO.IN)

    @property
    def name(self):
        return self.__name

    @property
    def min_range(self):
        return self.__min_range

    @property
    def max_range(self):
        return self.__max_range

    @property
    def field_of_view(self):
        return self.__field_of_view

    def get_range(self):
        # generate 0.01 ms impulse
        GPIO.output(self.__gpio_trigger, GPIO.HIGH)
        time.sleep(0.00001)
        GPIO.output(self.__gpio_trigger, GPIO.LOW)

        start_time = time.time()
        stop_time = time.time()
        iteration_start_time = time.time()

        # save start time
        while GPIO.input(self.__gpio_echo) == 0:
            start_time = time.time()
            if start_time - iteration_start_time > self.__max_wait_time:
                return math.nan

        # save time of arrival
        while GPIO.input(self.__gpio_echo) == 1:
            stop_time = time.time()
            if stop_time - iteration_start_time > self.__max_wait_time:
                return self.__max_range

        # time difference between start and stop time
        time_elapsed = stop_time - start_time
        # multiply with the half sonic speed
        range_m = time_elapsed * HALF_SONIC_SPEED

        if range_m > self.__max_range:
            range_m = self.__max_range
            
        if range_m < self.__min_range:
            range_m = self.__min_range
            
        return range_m

To avoid any damage, set values for parameters gpio_trigger and gpio_echo carefully. **Current values are examples and valid for my robot only**

In [6]:
try:
    sonar_right = UltraSonicSonar("right", 15, 14, 0.05, 4.0, 0)
    sonar_right.init_hw()
    print("sonar name = %s, range [m] = %4.2f, " % (sonar_right.name, sonar_right.get_range()))

    sonar_center = UltraSonicSonar("center", 23, 24, 0.05, 4.0, 0)
    sonar_center.init_hw()
    print("sonar name = %s, range [m] = %4.2f, " % (sonar_center.name, sonar_center.get_range()))

    sonar_left = UltraSonicSonar("left", 25, 16, 0.05, 4.0, 0)
    sonar_left.init_hw()
    print("sonar name = %s, range [m] = %4.2f, " % (sonar_left.name, sonar_left.get_range()))
finally:
    GPIO.cleanup()

sonar name = right, range [m] = 0.57, 
sonar name = center, range [m] = 0.58, 
sonar name = left, range [m] = 0.06, 
